In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
data = pd.read_csv("Startups.csv")

In [4]:
data.head()

,R&D Spend,Administration,Marketing Spend,Profit
0,165349.20,136897.80,471784.10,192261.83
1,162597.70,151377.59,443898.53,191792.06
2,153441.51,101145.55,407934.54,191050.39
3,144372.41,118671.85,383199.62,182901.99
4,142107.34,91391.77,366168.42,166187.94


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
R&D Spend          50 non-null float64
Administration     50 non-null float64
Marketing Spend    50 non-null float64
Profit             50 non-null float64
dtypes: float64(4)
memory usage: 1.7 KB


In [7]:
data.describe()

,R&D Spend,Administration,Marketing Spend,Profit
count,50.000000,50.000000,50.000000,50.000000
mean,73721.615600,121344.639600,211025.097800,112012.639200
std,45902.256482,28017.802755,122290.310726,40306.180338
min,0.000000,51283.140000,0.000000,14681.400000
25%,39936.370000,103730.875000,129300.132500,90138.902500
50%,73051.080000,122699.795000,212716.240000,107978.190000
75%,101602.800000,144842.180000,299469.085000,139765.977500
max,165349.200000,182645.560000,471784.100000,192261.830000


In [19]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)

In [20]:
train.columns

Index(['R&D Spend', 'Administration', 'Marketing Spend', 'Profit'], dtype='object')

In [67]:
train_x = train[['R&D Spend', 'Administration', 'Marketing Spend']]
train_y = train['Profit']

test_x = test[['R&D Spend', 'Administration', 'Marketing Spend']]
test_y = test['Profit']

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipline = Pipeline([
    ('std_scale',StandardScaler())
])

train_x_prepared = num_pipline.fit_transform(train_x)

In [31]:
from sklearn.tree import DecisionTreeRegressor

tree_reg0 = DecisionTreeRegressor(random_state=42)
tree_reg0.fit(train_x_prepared, train_y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [99]:
# RMSE for the model
from sklearn.metrics import mean_squared_error

age_predictions = tree_reg0.predict(train_x_prepared)
tree_mse = mean_squared_error(train_y, age_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

In [100]:
# RMSE using Cross-Validation 

from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg0, train_x_prepared, train_y,
                         scoring="neg_mean_squared_error", cv=2)
tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

array([16859.226356  , 14606.77938502])

In [34]:
# Overfitting

In [101]:
# Reqularize the model by setting optimal values for hyperparamters

from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 16 (4×4) combinations of hyperparameters
    {'min_samples_leaf': [1,2,3], 'max_features': [1,2,3]}
  ]

grid_search = GridSearchCV(tree_reg0, param_grid, cv=2,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
# Fit Grid Search CV
grid_search.fit(train_x_prepared, train_y)

GridSearchCV(cv=2, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=42, splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'max_features': [1, 2, 3],
                          'min_samples_leaf': [1, 2, 3]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             sc

In [102]:
grid_search.best_params_

{'max_features': 3, 'min_samples_leaf': 2}

In [103]:
# Building the model with best hyperparamters

from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(random_state=42, max_depth=3,min_samples_leaf=2)
tree_reg1.fit(train_x_prepared, train_y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=3,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [104]:
# RMSE forthe model

age_predictions = tree_reg1.predict(train_x_prepared)
tree_mse = mean_squared_error(train_y, age_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

6208.890762723904

In [105]:
# RMSE using Cross-Validation 
scores = cross_val_score(tree_reg1, train_x_prepared, train_y,
                         scoring="neg_mean_squared_error", cv=4)
tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

array([18162.72828774, 12438.12835425, 16720.03792685, 11624.15677321])

In [45]:
# It is still overfitting have to tweek the hyper paramter again

In [106]:
# Simple Linear Regression

In [107]:
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()
linear_reg.fit(train_x_prepared, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [108]:
# RMSE forthe model
age_predictions = linear_reg.predict(train_x_prepared)
reg_mse = mean_squared_error(train_y, age_predictions)
reg_rmse = np.sqrt(reg_mse)
reg_rmse

8740.004883086891

In [109]:
# RMSE using Cross-Validation 
scores = cross_val_score(linear_reg, train_x_prepared, train_y,
                         scoring="neg_mean_squared_error", cv=3)
reg_rmse_scores = np.sqrt(-scores)
reg_rmse_scores

array([ 8937.05386393, 13803.90346569,  6753.11601616])

In [110]:
# Average
np.sum(reg_rmse_scores)/ reg_rmse_scores.shape

array([9831.35778193])

In [111]:
# Good 

In [112]:
# Testing

test_x_prep = num_pipline.fit_transform(test_x)

In [113]:
predicted = linear_reg.predict(test_x_prep)

In [114]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(predicted,test_y)
np.sqrt(mse)

19316.35374498761

In [115]:
# Overfitting 

-------------